In [1]:
import pandas as pd
UserProfile=pd.read_csv('weiboUserProfile_csv.txt',header=None,sep=':')
UserProfile.columns=['User_id','Place','Gender','Year','Horoscope']
UserProfile.head()

C:\Users\admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,User_id,Place,Gender,Year,Horoscope
0,1381078220,Shanghai Putuo District,female,NaN,NaN
1,1723807154,"Xuhui District, Shanghai",male,NaN,Leo
2,1500165530,Shanghai Pudong New Area,female,1989,NaN
3,1246687181,Shanghai Pudong New Area,female,NaN,Aquarius
4,1871921945,Shanghai,female,1982,NaN


In [2]:
import pandas as pd
CheckinInfo=pd.read_csv('checkin_bj_uniq.txt',header=None,sep='\t')
CheckinInfo.columns=['User_id','Time','latitude','longititude','Item_id']
CheckinInfo.head()

,User_id,Time,latitude,longititude,Item_id
0,1000030044,2011/10/11 17:01:59,36.677125,117.023793,j:3D6A0B8EF0FB83BD
1,1000030044,2011/10/15 14:33:07,40.167353,116.565249,j:351926FDF2BF794A
2,1000030044,2011/10/20 16:47:18,39.895401,116.322000,j:EB34D7A2DCA7F021
3,1000030044,2011/10/26 12:38:23,38.043217,114.481255,j:7C22E78A6E6BC3FFD7807B30A00D0CF2
4,1000030044,2011/10/29 10:33:20,39.953124,116.492071,j:8DFC0EDFDD5E6858


In [3]:
CheckinInfo["User_id"]=CheckinInfo["User_id"].astype('int64')
UserProfile["User_id"]=UserProfile["User_id"].astype('int64')
Checkin_user=pd.merge(CheckinInfo,UserProfile,on='User_id')
Checkin_user.head()

,User_id,Time,latitude,longititude,Item_id,Place,Gender,Year,Horoscope
0,1000030044,2011/10/11 17:01:59,36.677125,117.023793,j:3D6A0B8EF0FB83BD,"Chaoyang District, Beijing",male,NaN,Cancer
1,1000030044,2011/10/15 14:33:07,40.167353,116.565249,j:351926FDF2BF794A,"Chaoyang District, Beijing",male,NaN,Cancer
2,1000030044,2011/10/20 16:47:18,39.895401,116.322000,j:EB34D7A2DCA7F021,"Chaoyang District, Beijing",male,NaN,Cancer
3,1000030044,2011/10/26 12:38:23,38.043217,114.481255,j:7C22E78A6E6BC3FFD7807B30A00D0CF2,"Chaoyang District, Beijing",male,NaN,Cancer
4,1000030044,2011/10/29 10:33:20,39.953124,116.492071,j:8DFC0EDFDD5E6858,"Chaoyang District, Beijing",male,NaN,Cancer


In [8]:
Checkin_user["Year"]=Checkin_user["Year"].astype(float)
Checkin_user["Year"]=Checkin_user["Year"].fillna(0).astype(int)

a1=Checkin_user.groupby( [ "Gender", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a2=Checkin_user.groupby( [ "Place", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a3=Checkin_user.groupby( [ "Year", "Item_id"] ).size().to_frame(name = 'count').reset_index()
a4=Checkin_user.groupby( [ "Horoscope", "Item_id"] ).size().to_frame(name = 'count').reset_index()


In [9]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

person_c_1 = CategoricalDtype(sorted(a1.Gender.unique()), ordered=True)
person_c_2 = CategoricalDtype(sorted(a2.Place.unique()), ordered=True)
person_c_3 = CategoricalDtype(sorted(a3.Year.unique()), ordered=True)
person_c_4 = CategoricalDtype(sorted(a4.Horoscope.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(CheckinInfo.Item_id.unique()), ordered=True)

row_1 = a1.Gender.astype(person_c_1).cat.codes
row_2 = a2.Place.astype(person_c_2).cat.codes
row_3 = a3.Year.astype(person_c_3).cat.codes
row_4 = a4.Horoscope.astype(person_c_4).cat.codes
col_1 = a1.Item_id.astype(thing_c).cat.codes
col_2 = a2.Item_id.astype(thing_c).cat.codes
col_3 = a3.Item_id.astype(thing_c).cat.codes
col_4 = a4.Item_id.astype(thing_c).cat.codes

sparse_checkin_gender = csr_matrix((a1['count'], (row_1, col_1)),shape=(person_c_1.categories.size, thing_c.categories.size))
sparse_checkin_place = csr_matrix((a2['count'], (row_2, col_2)),shape=(person_c_2.categories.size, thing_c.categories.size))
sparse_checkin_year = csr_matrix((a3['count'], (row_3, col_3)),shape=(person_c_3.categories.size, thing_c.categories.size))
sparse_checkin_horoscope = csr_matrix((a4['count'], (row_4, col_4)),shape=(person_c_4.categories.size, thing_c.categories.size))


In [10]:
print(sparse_checkin_gender.shape)
print(sparse_checkin_place.shape)
print(sparse_checkin_year.shape)
print(sparse_checkin_horoscope.shape)


(2, 213673)
(389, 213673)
(59, 213673)
(12, 213673)


In [7]:
Checkin_user["Year"]=Checkin_user["Year"].astype(float)
Checkin_user["Year"]=Checkin_user["Year"].fillna(0).astype(int)